In [80]:
import mhcflurry, seaborn, numpy, pandas, pickle, sklearn, collections, scipy, time
import mhcflurry.dataset
import fancyimpute, locale
from matplotlib import pyplot


import sklearn.metrics
import sklearn.cross_validation
%matplotlib inline

from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)


def print_full(x):
    pandas.set_option('display.max_rows', len(x))
    print(x)
    pandas.reset_option('display.max_rows')

In [185]:
impute_comparison = pandas.read_csv("../data/impute_comparison.csv")

pyplot.figure(figsize=(4,4.5))
seaborn.set_style("whitegrid")
column = "test_f1"
for (i, (allele, data)) in enumerate(impute_comparison.groupby("allele")):
    pyplot.subplot(3,1, i + 1, sharey=pyplot.gca())
    grouped = data.groupby(["train_size", "impute"])[column].mean().to_frame().reset_index()
    grouped["std_error"] = data.groupby(["train_size", "impute"])[column].std().to_frame().reset_index()[column]
    for impute in [True, False]:
        sub = grouped[grouped.impute == impute]
        pyplot.errorbar(
                       x=sub.train_size.values,
                       y=sub[column].values,
                       yerr=sub.std_error.values,
                       label=("with" if impute else "without") + " imputation",
                       color=seaborn.get_color_cycle()[0] if impute else seaborn.get_color_cycle()[1])    
    pyplot.title(allele)
    pyplot.xscale("log")
    if i == 0:
        #pyplot.ylabel("AUC")
        pyplot.legend(loc='lower right', fancybox=True, frameon=True)

    if i == 2:
        pyplot.xlabel("Training set size")
    pyplot.savefig("../figures/impute_comparison.pdf")

#pyplot.legend(loc='lower right', ncol=1, bbox_to_anchor=(2.2,.1))

pyplot.tight_layout()


In [177]:
scores_df = pandas.read_csv("../data/validation_scores.csv", index_col="allele")
scores_df
big_alleles = list((scores_df.ix[(scores_df.index != "overall") & (scores_df.train_size >= 500)]).index)
small_alleles = list((scores_df.ix[(scores_df.index != "overall") & (scores_df.train_size < 500)]).index)
small_alleles

def sub_df(name):
    result = scores_df[[c for c in list(scores_df.columns) if (name + "_") in c]].copy()
    result.columns = [c.split("_")[-1] for c in list(result.columns)]
    return result

predictors = ["mhcflurry ensemble all", "mhcflurry 0", "netmhc", "netmhcpan", "smmpmbec_cpp"]

d = dict((name, sub_df(name)) for name in predictors)
panel_all = pandas.Panel(d)
panel = panel_all[predictors]
panel

name_adjustments = {
    "tau": "Kendall's $\tau$",
    "auc": "AUC",
    "f1": "$F_1$ score",
    "mhcflurry ensemble all": "mhcflurry ensemble",
    "mhcflurry 0": "mhcflurry single",
    "smmpmbec_cpp": "smmpmbec cpp",
}

def latex_table(df, max_df=None, caption=None, label=None, name_adjustments=name_adjustments):
    if max_df is None:
        max_df = df
    df_strings = df.copy()
    for col in df.columns:
        del df_strings[col]
        renamed_col = name_adjustments.get(col, col)
        df_strings[renamed_col] = [
            ("\\textbf{%s}" if x == max_df[col].max() else "%s") % (("%0.4f" % x) if isinstance(x, float) else x)
            for x in df[col]
        ]
    df_strings.index = [name_adjustments.get(x, x) for x in df_strings.index]

    print(r"\begin{table}[h]")
    print(r"\centering")
    print(df_strings.to_latex(escape=False))
    if caption:
        print("\\caption{%s}" % caption.strip())
    if label:
        print("\\label{tab:%s}" % label.strip())
    print(r"\end{table}")
    print("")

    
latex_table(panel[:,"overall"].T, label="measurementweighted",
            caption="""
            Performance on BLIND with equal weight given to each measurement ({:,d} measurements)
            """.format(scores_df.test_size.sum()))
latex_table(panel.mean(1).T, label="alleleweighted",
            caption="""
            Performance on BLIND with equal weight given to each allele (%d alleles)
            """ % panel.shape[1])
latex_table(panel[:,big_alleles].mean(1).T, label="alleleweightedbig",
            caption="""
            Performance on BLIND for alleles with at least 500 training observations (%d alleles)
            """ % len(big_alleles))
#latex_table(panel[:,small_alleles].mean(1).T,
#            caption="""
#            Performance on BLIND for alleles with fewer than 500 training observations (%d alleles)
#            """ % len(small_alleles))



df=panel[:,small_alleles,"auc"]
df["train size"] = scores_df.ix[scores_df.index != 'overall'].train_size.apply(lambda x: str(int(x)))
df["test size"] = scores_df.ix[scores_df.index != 'overall'].test_size.apply(lambda x: str(int(x)))
df = df[["train size", "test size"] + list(panel.items)]
df.sort("train size", inplace=True)
latex_table(df.T, max_df = df.T.iloc[2:], label="smallalleles", caption="""
AUC scores on BLIND dataset for alleles with fewer than 500 training examples
""")

\begin{table}[h]
\centering
\begin{tabular}{llll}
\toprule
{} &              AUC &      $F_1$ score & Kendall's $\tau$ \\
\midrule
mhcflurry ensemble &           0.9325 &           0.7847 &  \textbf{0.5865} \\
mhcflurry single   &           0.9313 &           0.7831 &           0.5843 \\
netmhc             &           0.9323 &  \textbf{0.8072} &           0.5863 \\
netmhcpan          &  \textbf{0.9326} &           0.7996 &           0.5814 \\
smmpmbec cpp       &           0.9213 &           0.7903 &           0.5649 \\
\bottomrule
\end{tabular}

\caption{Performance on BLIND with equal weight given to each measurement (53,776 measurements)}
\label{tab:measurementweighted}
\end{table}

\begin{table}[h]
\centering
\begin{tabular}{llll}
\toprule
{} &              AUC &      $F_1$ score & Kendall's $\tau$ \\
\midrule
mhcflurry ensemble &           0.9103 &           0.6635 &           0.4890 \\
mhcflurry single   &           0.9095 &           0.6551 &           0.4874 \\
netmhc          

53776